In [3]:
import pandas as pd
import nltk
from nltk import pos_tag, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import string
from tqdm import tqdm
from spellchecker import SpellChecker

You have been provided with a dataset of over 350,000 such complaints for 5 common issue types. 

> Your goal is to *train* a **text classification** model to **identify** the **issue type** based on the consumer **complaint narrative**.

## Data Loading

In [4]:
# Read in data
complaints = pd.read_csv('../data/complaints.csv')

# Standardized column naming
complaints.columns = [x.replace('Consumer ', '').lower().replace(' ', '_') for x in complaints.columns]
complaints['complaint_length'] = complaints['complaint_narrative'].apply(len)

# Explore the count of unique complaints for each issue.
complaints['issue'].value_counts()

Incorrect information on your report    229305
Attempts to collect debt not owed        73163
Communication tactics                    21243
Struggling to pay mortgage               17374
Fraud or scam                            12347
Name: issue, dtype: int64

It appears that the data has several entries redacted due to personally idenfiyiable information.  We will write a function to remove cases of text redaction.

In [5]:
complaints.head()

,complaint_narrative,issue,complaint_length
0,My name is XXXX XXXX this complaint is not mad...,Incorrect information on your report,711
1,I searched on XXXX for XXXXXXXX XXXX and was ...,Fraud or scam,1958
2,I have a particular account that is stating th...,Incorrect information on your report,294
3,I have not supplied proof under the doctrine o...,Attempts to collect debt not owed,3444
4,Hello i'm writing regarding account on my cred...,Incorrect information on your report,876


Explore some data from the narratives column.


In [6]:
#there appears to be multiple areas where data is redacted.
complaints['complaint_narrative']

0         My name is XXXX XXXX this complaint is not mad...
1         I searched on XXXX for XXXXXXXX XXXX  and was ...
2         I have a particular account that is stating th...
3         I have not supplied proof under the doctrine o...
4         Hello i'm writing regarding account on my cred...
                                ...                        
353427           Collections account I have no knowledge of
353428    Dear CFPB Team, The reason for my complaint is...
353429    FRCA violations : Failing to Follow Debt Dispu...
353430    My Father, a XXXX XXXX  acquired an HECM rever...
353431    I have tried to contact cash app about a fraud...
Name: complaint_narrative, Length: 353432, dtype: object

In [7]:
# Looking at some narratives inside the "Incorrect information" complaint.
seed = 123
for complaint in complaints.loc[complaints['issue'] == 'Incorrect information on your report', 'complaint_narrative'].sample(3, random_state=seed):
    print(complaint)
    print('-----------------------------')

I just reviewed a copy of my Experian credit report and found the following information to be an error. I am a victim of identity theft and did not make these charges. I reported the theft of my identity to the Federal Trade Commission.
-----------------------------
after my legal separation from my husband he started to open credit in my name with no permission I have a legal case against him but can not find him he has ruined my life his name is XXXX XXXX XXXX   remove I have disputed by mail with all three credit bureaus. act # XXXX owing an alleged {$2200.00} XXXX   XXXX  orig creditor XXXX
-----------------------------
Equifax report a collections account " XXXX XXXX XXXX ''. I have no knowledge of this account. I have not been furnished any proof or verifications of this account. I don't have any signed contract agreements with XXXX XXXX XXXX, verbal agreements nor any paperwork associated with them. A proper investigation according to FCRA was not conducted, word of mouth from t

Encode "issues" as numerical values

In [8]:
# Issue mapping dictionary to apply to columns to encode as numerical values
issue_mapping = {'Attempts to collect debt not owed': 1, 
                 'Communication tactics': 2, 
                 'Fraud or scam': 3, 
                 'Incorrect information on your report': 4,
                 'Struggling to pay mortgage': 5}

# replace the issue values with numerical values
complaints['issue'] = complaints['issue'].replace(issue_mapping)
complaints.head(n=5)

,complaint_narrative,issue,complaint_length
0,My name is XXXX XXXX this complaint is not mad...,4,711
1,I searched on XXXX for XXXXXXXX XXXX and was ...,3,1958
2,I have a particular account that is stating th...,4,294
3,I have not supplied proof under the doctrine o...,1,3444
4,Hello i'm writing regarding account on my cred...,4,876


## Text Cleaning

### Cleaning complaints_narrative

In [9]:
# Replacing strings in narrative to remove consecutive x's
complaints['complaint_narrative'] = complaints['complaint_narrative'].str.replace(r'[xX][xX]+','', regex=True)

# Stripping all numeric values from complaints
complaints['complaint_narrative'] = complaints['complaint_narrative'].str.replace('\d+', '', regex=True)

# Lowercasing text
complaints['complaint_narrative'] = complaints['complaint_narrative'].apply(lambda x: x.lower())

# Removing punctuation and special characters
translator = str.maketrans('', '', string.punctuation)
complaints['complaint_narrative'] = complaints['complaint_narrative'].apply(lambda x: x.translate(translator))

### Applying stopword removal to complaints narratives.

In [10]:
# define stopwords
stop_words = set(stopwords.words('english'))

# apply stopword removal to the 'complaint_narrative' column
complaints['complaint_narrative'] = complaints['complaint_narrative'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

### Tokenization of complaint_narrative (Warning: 6 minute runtime)

In [12]:
# Tokenizing text (Careful! 6 min run time)
complaints['tokens'] = list(tqdm(complaints['complaint_narrative'].apply(nltk.word_tokenize), desc="Tokenizing"))

Tokenizing: 100%|██████████| 353432/353432 [00:00<00:00, 2736830.03it/s]


### Stemming Text (Warning: 8 Minute run time)

In [10]:
# Instantiate the PorterStemmer
stemmer = PorterStemmer()

# Stemming on the existing 'tokens' column and create a new 'stemmed_tokens' column
complaints['stems'] = list(tqdm(complaints['tokens'].apply(lambda x: [stemmer.stem(token) for token in x]), desc="Stemming"))

Stemming: 100%|██████████| 353432/353432 [00:00<00:00, 2786400.43it/s]


### Lemmatizing Text

In [11]:
# Instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize tokens
complaints['lemmas'] = list(tqdm(complaints['tokens'].apply(lambda x:[lemmatizer.lemmatize(word) for word in x]), desc="Lemmatizing"))

Lemmatizing: 100%|██████████| 353432/353432 [00:00<00:00, 2952878.78it/s]


In [12]:
complaints.head(n=2)

,complaint_narrative,issue,complaint_length,tokens,stems,lemmas
0,name complaint made error neither made third p...,4,711,"[name, complaint, made, error, neither, made, ...","[name, complaint, made, error, neither, made, ...","[name, complaint, made, error, neither, made, ..."
1,searched pointed website legitimately believed...,3,1958,"[searched, pointed, website, legitimately, bel...","[search, point, websit, legitim, believ, websi...","[searched, pointed, website, legitimately, bel..."


### Bread and Butter 🥒🫙

In [ ]:
# Drop the "complaints_narrative" column
complaints = complaints.drop('complaint_narrative', axis=1)

# Path and filename for the output .pkl file
canning = '../data/complaints_nlp.pkl'

# Save the .pkl, it's quite a "dill"!
complaints.to_pickle(canning)

print("DataFrame successfully pickled! It's now a 'pickle'-icious file!")